In [11]:
!source /etc/profile

import os
import sys
import random
import time
from collections import deque
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Activation, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import json
import joblib

# Check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.list_physical_devices()

Num GPUs Available:  1


In [2]:
SEQ_LEN = 24 #hours
FUTURE_PERIOD_PREDICT = 4 #hours

csv_file = "data/formatted/ETHUSDT-1h-data.csv"
model_path = "models/TANH-e20-b64-s24-fpp4-1643955277-e05-vacc0.000-1643955284.model"

In [3]:
## Import data
# DATA MUST BE FORMATTED USING CSV_FORMATTER.IPYNB



data = pd.read_csv(csv_file, skiprows=[0], names=["timestamp", "open", "high", "low", "close", "volume", "rsi", "ema"])

data.set_index("timestamp", inplace=True)

data.head()

,open,high,low,close,volume,rsi,ema
timestamp,,,,,,,
1503064800,304.73,311.06,304.73,308.88,332.56430,56.653853,305.268933
1503068400,308.88,311.79,306.27,307.78,261.15170,55.497845,305.393589
1503072000,307.78,310.29,305.73,308.47,388.17208,56.079385,305.543699
1503075600,307.17,309.66,301.30,303.22,357.19041,50.849211,305.432161
1503079200,303.22,303.22,296.32,298.52,398.64644,46.806430,305.105516


In [4]:
## Formatting data

# def classify(current, future):
#     return float((future - current) / current)


# data["future"] = data["close"].shift(-FUTURE_PERIOD_PREDICT)

# Cut off NaNs
# data = data[:-FUTURE_PERIOD_PREDICT]
data.dropna(inplace=True)

# data["target"] = list(map(classify, data["close"], data["future"]))
# data[["close", "future", "target"]].tail()
# data = data.drop("future", 1)

# Split dataset
# last_5_pct = int(len(data) * .95)

# train_data = data[:last_5_pct]
# validation_data = data[last_5_pct:]

# print(f"{len(train_data)} :: {len(validation_data)}")

data.head()

,open,high,low,close,volume,rsi,ema
timestamp,,,,,,,
1503064800,304.73,311.06,304.73,308.88,332.56430,56.653853,305.268933
1503068400,308.88,311.79,306.27,307.78,261.15170,55.497845,305.393589
1503072000,307.78,310.29,305.73,308.47,388.17208,56.079385,305.543699
1503075600,307.17,309.66,301.30,303.22,357.19041,50.849211,305.432161
1503079200,303.22,303.22,296.32,298.52,398.64644,46.806430,305.105516


In [5]:
## Preprocess Data

def preprocess_df(df):
    temp_df = pd.DataFrame()
    for col in df.columns:
        temp_df[col] = df[col]

    for col in temp_df.columns:
        temp_df[col] = temp_df[col].pct_change()
        temp_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        temp_df.dropna(inplace=True)

        temp_df[col] = preprocessing.StandardScaler().fit_transform(temp_df[col].values.reshape(-1,1))

    temp_df.dropna(inplace=True)

    sequential_data = []
    prev_periods = deque(maxlen=SEQ_LEN)

    for i in temp_df.values:
        prev_periods.append([n for n in i])
        if len(prev_periods) == SEQ_LEN:
            sequential_data.append([np.array(prev_periods)])

    # random.shuffle(sequential_data)

    # Balance buys and sells
    # buys = []
    # sells = []

    # for seq, target in sequential_data:
    #     if target == 0:
    #         sells.append([seq, target])
    #     elif target == 1:
    #         buys.append([seq, target])

    # lower = min(len(buys), len(sells))

    # buys = buys[:lower]
    # sells = sells[:lower]

    # sequential_data = buys + sells

    # random.shuffle(sequential_data)

    X = [d[0] for d in sequential_data]
    # Y = [d[1] for d in sequential_data]

    return np.array(X)

    # return sequential_data
    

seq_data = preprocess_df(data)

In [6]:
## Load model

model = load_model(model_path)
model.summary()


2022-02-04 07:56:04.165690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-04 07:56:04.166176: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-04 07:56:04.166585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-04 07:56:04.509969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-04 07:56:04.510382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 24, 128)           69632     
                                                                 
 dropout_12 (Dropout)        (None, 24, 128)           0         
                                                                 
 batch_normalization_9 (Batc  (None, 24, 128)          512       
 hNormalization)                                                 
                                                                 
 lstm_10 (LSTM)              (None, 24, 128)           131584    
                                                                 
 dropout_13 (Dropout)        (None, 24, 128)           0         
                                                                 
 batch_normalization_10 (Bat  (None, 24, 128)          512       
 chNormalization)                                     

In [9]:
bal = 10000
usd_bal = 10000
coin_bal = 0

price = 234890

list(data["close"])[0+24]

# print(f"BAL {bal}\tUSD {usd_bal}\tCOIN {coin_bal}")

287.14

In [13]:
with tf.device("/device:GPU:0"):
    print(json.dumps([float(list(d)[0]) for d in list(model.predict(seq_data))], indent=4))

[
    -0.000844615395180881,
    -0.001002763514406979,
    -0.0008752259891480207,
    -0.0008055774960666895,
    -0.0007466186652891338,
    -0.0007916520698927343,
    -0.0007460789056494832,
    -0.0007340976735576987,
    -0.0007957079797051847,
    -0.0008248243830166757,
    -0.0007743319147266448,
    -0.0007134424522519112,
    -0.0008000725065357983,
    -0.0007417299784719944,
    -0.0007213188800960779,
    -0.0007172869518399239,
    -0.000697849493008107,
    -0.0006752976914867759,
    -0.00064606755040586,
    -0.000612997158896178,
    -0.0005889582680538297,
    -0.0005805252585560083,
    -0.0005729880067519844,
    0.0017784201772883534,
    0.005240114405751228,
    -0.000657855998724699,
    -0.0016956300241872668,
    -0.0021964581683278084,
    -0.002929974813014269,
    -0.0034954845905303955,
    -0.004174583125859499,
    -0.004454304464161396,
    -0.0038840691559016705,
    -0.0028888657689094543,
    -0.002042879816144705,
    -0.0012512411922216415,
    

In [7]:
## Trading bot (test only)

bal = 10000
usd_bal = bal
coin_bal = 0

model_trade_x = list(range(len(seq_data) + 1))
model_trade_y = []

def do_buy(price):
    global usd_bal
    global coin_bal

    if usd_bal == 0:
        return

    usd_bal -= usd_bal * calc_fee()
    coin_bal += (usd_bal / price)
    usd_bal = 0

    calc_bal(price)

def do_sell(price):
    global usd_bal
    global coin_bal

    if coin_bal == 0:
        return

    usd_bal += (coin_bal * price)
    usd_bal -= usd_bal * calc_fee()
    coin_bal = 0

    calc_bal(price)

def calc_bal(price):
    global bal
    bal = usd_bal + (coin_bal * price)

def calc_fee():
    return 0
    # if bal > 1000000:
    #     return .0018
    # elif bal > 100000:
    #     return .0020
    # elif bal > 50000:
    #     return .0025
    # elif bal > 10000:
    #     return .0035
    # else:
    #     return .0050

def get_price(i):
    return list(data["close"])[i+SEQ_LEN]

def predict(seq):
    with tf.device("/device:GPU:0"):
        return model.predict(seq)[0]

def full_predict(full_seq):
    with tf.device("/device:GPU:0"):
        return model.predict(full_seq)


def main_loop_iter():
    print(f"ITER LEN: {len(seq_data)}")

    for i in range(len(seq_data)):
        print(f"SEQ: {i}\tBAL: {bal}")
        model_trade_y.append(bal)
        prediction = predict(seq_data[i:i+1])

        if prediction[0] > prediction[1]:
            do_buy(get_price(i))
        else:
            do_sell(get_price(i))
        

    do_sell(get_price(len(seq_data)-1))
    model_trade_y.append(bal)
    print(f"FINAL BAL: {bal}")

def main_loop():
    print(f"ITER LEN: {len(seq_data)}")

    prediction = full_predict(seq_data)

    for i in range(len(seq_data)):
        print(f"SEQ: {i}\tBAL: {bal}")
        model_trade_y.append(bal)

        buy_indicator = prediction[i][0]
        sell_indicator = prediction[i][1]

        # if prediction[i][0] > .8 and prediction[i][1] < .2:
        #     do_buy(get_price(i))
        # if prediction[i][1] > .8 and prediction[i][0] < .2:
        #     do_sell(get_price(i))
        # else:
        #     pass

        if buy_indicator > sell_indicator:
            do_buy(get_price(i))
        else:
            do_sell(get_price(i))
            
        

    do_sell(get_price(len(seq_data)-1))
    model_trade_y.append(bal)
    print(f"FINAL BAL: {bal}")


main_loop()

ITER LEN: 38543


2022-02-03 03:06:59.733310: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


SEQ: 0	BAL: 10000
SEQ: 1	BAL: 10000.0
SEQ: 2	BAL: 10000.0
SEQ: 3	BAL: 10047.714121349532
SEQ: 4	BAL: 10047.714121349532
SEQ: 5	BAL: 10047.714121349532
SEQ: 6	BAL: 10076.738953985549
SEQ: 7	BAL: 10076.738953985549
SEQ: 8	BAL: 10076.738953985549
SEQ: 9	BAL: 10148.458306319453
SEQ: 10	BAL: 10148.458306319453
SEQ: 11	BAL: 10148.458306319453
SEQ: 12	BAL: 10148.458306319453
SEQ: 13	BAL: 10148.458306319453
SEQ: 14	BAL: 10148.458306319453
SEQ: 15	BAL: 10265.554589128602
SEQ: 16	BAL: 10265.554589128602
SEQ: 17	BAL: 10214.39848097542
SEQ: 18	BAL: 10214.39848097542
SEQ: 19	BAL: 10214.39848097542
SEQ: 20	BAL: 10214.39848097542
SEQ: 21	BAL: 10214.39848097542
SEQ: 22	BAL: 10849.711242923699
SEQ: 23	BAL: 10849.711242923699
SEQ: 24	BAL: 10849.711242923699
SEQ: 25	BAL: 10983.75539203732
SEQ: 26	BAL: 10983.75539203732
SEQ: 27	BAL: 10983.75539203732
SEQ: 28	BAL: 10983.75539203732
SEQ: 29	BAL: 10983.75539203732
SEQ: 30	BAL: 10983.75539203732
SEQ: 31	BAL: 10983.75539203732
SEQ: 32	BAL: 10983.75539203732
SE

In [9]:
plt.rcParams["figure.figsize"] = (15,15)
plt.plot(model_trade_x, model_trade_y)
plt.plot(model_trade_x, data["close"].tail(38568))

NameError: name 'model_trade_x' is not defined